# Yoon Kim

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

import torchtext

from tensorboardX import SummaryWriter
from tqdm import tqdm as tqdm

np.random.seed(42)
CUDA = torch.cuda.is_available()

CUDA

True

In [4]:
# alphabet from the paper
# https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf
ALPHABET = ['\n'] + [s for s in """ abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&* ̃‘+-=<>()[]{}"""]

MAX_WORD_LEN = 16  # chars in word (try 32?)
MAX_TEXT_LEN = 256  # words in text

# Data preparation

Чтобы использовать CNN на слова, нужно фиксировать длину слова.

# Model

In [5]:
class YoonKimModel:
    def __init__(self, n_filters, cnn_kernel_size, pool_kernel_size, pool_stride,
                 hidden_dim_2, dropout=0.5, init_function=None, embedding_dim=len(ALPHABET)):
        super(YoonKimModel, self).__init__()
        self.dropout = dropout
        self.init_function = init_function
        self.embedding_dim = embedding_dim
        self.n_filters = n_filters
        self.cnn_kernel_size = cnn_kernel_size
        self.pool_kernel_size = pool_kernel_size
        self.pool_stride = pool_stride
        self.hidden_dim_2 = hidden_dim_2

        self.embedding = nn.Linear(len(ALPHABET), embedding_dim)
        self.chars_cnn = nn.Sequential(
            nn.Conv1d(embedding_dim, n_filters, kernel_size=cnn_kernel_size, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_stride)
        )
        if init_function is not None:
            self.conv[0].weight = init_function(self.conv[0].weight)

        self.conv_dim = n_filters * int(((MAX_WORD_LEN - cnn_kernel_size) / 1 - pool_kernel_size) / pool_stride + 1)

        self.words_rnn = nn.GRU(conv_dim, hidden_dim_2, dropout=dropout)
        self.projector = nn.Linear(hidden_dim_2, 2)
        
    def forward(self, x):
        # как избавиться от зависимости от BATCH_SIZE?
        words_tensor = torch.zeros(MAX_TEXT_LEN, BATCH_SIZE, self.conv_dim)
        for i, word in enumerate(x[:MAX_TEXT_LEN]):
            word = self.embedding(word)
            word = self.chars_cnn(word)
            word = word.view(word.size(0), -1)
            words_tensor[i, :] = word

        x, _ = self.words_rnn(words_tensor)
        x = self.projector(x[-1])
        return x

#     def describe(self):
#         return 'YoonKimModel_filters_%s_cnn_%s_pool_%s_stride_%s'

In [7]:
model = YoonKimModel(n_filters=256, cnn_kernel_size=5, pool_kernel_size=4, pool_stride=3, dropout=0.5)

NameError: name 'hidden_dim_2' is not defined